<a href="https://colab.research.google.com/github/mayssamhira/SummaText-Smart-Abstractive-Summaries-Using-Transformers/blob/main/SummaText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install & Import Libraries**

In [1]:
!pip install -q transformers datasets rouge-score accelerate evaluate nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer
)
import evaluate
import torch

**Load Dataset**

In [3]:
ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [8]:
# Reduce size for Colab
ds = ds["train"].shuffle(seed=42).select(range(2000))
ds = ds.train_test_split(test_size=0.1)

**Load Tokenizer**

In [11]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

**Preprocessing**

In [12]:
def preprocess(batch):
    inputs = tokenizer(
        batch["dialogue"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    targets = tokenizer(
        batch["summary"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

**Tokenize Dataset**

In [13]:
#map function in dataframe
#df_source = ds.map(preprocess_function, batched = True)
tokenized_dataset = ds.map(
    preprocess,
    batched=True,
    remove_columns=ds["train"].column_names
)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [15]:
pip install -U transformers accelerate

In [16]:
import transformers
print(transformers.__version__)

4.57.3


In [17]:
evaluation_strategy="epoch"

In [18]:
evaluation_strategy="no"

In [19]:
training_args = TrainingArguments(
    output_dir="./bart-summarizer",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    fp16=True,
    num_train_epochs=2,
    logging_steps=50,
    save_total_limit=1,
    report_to="none"
)

In [21]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-4293983436.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
trainer.train()

Step,Training Loss
50,7.177600
100,3.048400
150,1.683900
200,0.933000
250,0.595800
300,0.485600
350,0.452200
400,0.448400
450,0.436900


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=450, training_loss=1.6957653003268771, metrics={'train_runtime': 302.7446, 'train_samples_per_second': 11.891, 'train_steps_per_second': 1.486, 'total_flos': 1097525624832000.0, 'train_loss': 1.6957653003268771, 'epoch': 2.0})

**ROUGE Metric**

In [26]:
!pip install rouge-metric
!pip install rouge_score
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 4.8 MB/s eta 0:00:00


In [27]:
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [28]:
save_path = "bart-summarizer-final"

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('bart-summarizer-final/tokenizer_config.json',
 'bart-summarizer-final/special_tokens_map.json',
 'bart-summarizer-final/vocab.json',
 'bart-summarizer-final/merges.txt',
 'bart-summarizer-final/added_tokens.json')

In [29]:
import shutil
shutil.make_archive("bart-summarizer-final", "zip", save_path)

'/content/bart-summarizer-final.zip'

In [30]:
MODEL_DIR = "bart-summarizer-final"

trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

('bart-summarizer-final/tokenizer_config.json',
 'bart-summarizer-final/special_tokens_map.json',
 'bart-summarizer-final/vocab.json',
 'bart-summarizer-final/merges.txt',
 'bart-summarizer-final/added_tokens.json')

In [31]:
!ls bart-summarizer-final

config.json		model.safetensors	 training_args.bin
generation_config.json	special_tokens_map.json  vocab.json
merges.txt		tokenizer_config.json
